Example 1: 1-parameter Persistence, vectorization and visualization



In [ ]:
from MPHvect.method import *
from persim import plot_diagrams

try:
    import multipers
except ImportError:
    print("Multipers not installed — some examples may fail.")

from ripser import ripser
import multipers.plots as mpp
from multipers.filtrations import RipsLowerstar, RipsCodensity
from multipers.filtrations.density import KDE

In [ ]:



n_points = 5000
points = np.random.uniform(low=-1, high=1, size=(n_points, 2))

# Plot the points
plt.figure(figsize=(6, 6))
plt.scatter(points[:, 0], points[:, 1], color='blue', s=10, alpha=0.6)
plt.title('1000 Random Points in the Plane (ℝ²)')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.axis('equal')  # Ensure aspect ratio is 1:1
plt.show()

diagrams = ripser(points)['dgms']

# Plot persistence diagram
plot_diagrams(diagrams, show=True)

In [ ]:
pers_diagram=diagrams[1]
MPHvect_visualize(pers_diagram)

Example 2: 2-parameter Persistence, vectorization and visualization


In [ ]:
#make sampling method


def sample_annulus(n_points, r_inner, r_outer, noise=0.0, center=(0,0)):
    """
    Sample points uniformly from an annulus with inner radius r_inner and outer radius r_outer.
    Optionally add Gaussian noise.
    """
    theta = np.random.uniform(0, 2*np.pi, n_points)
    r = np.sqrt(np.random.uniform(r_inner**2, r_outer**2, n_points))
    x = center[0] + r * np.cos(theta)
    y = center[1] + r * np.sin(theta)
    points = np.column_stack([x, y])

    if noise > 0:
        points += np.random.normal(scale=noise, size=points.shape)

    return points

def sample_disk(n_points, radius, noise=0.0, center=(0,0)):
    """
    Sample points uniformly from a disk with given radius.
    Optionally add Gaussian noise.
    """
    theta = np.random.uniform(0, 2*np.pi, n_points)
    r = np.sqrt(np.random.uniform(0, radius**2, n_points))
    x = center[0] + r * np.cos(theta)
    y = center[1] + r * np.sin(theta)
    points = np.column_stack([x, y])

    if noise > 0:
        points += np.random.normal(scale=noise, size=points.shape)

    return points

def generate_two_annuli_and_disk(n, r11,r12,r21, r22,r3,noise=0.05):
    """

    Generate points for two non-overlapping annuli and a disk.
    """
    area1=(np.pi*(r12**2-r11**2))
    area2=(np.pi*(r22**2-r21**2))
    area3=(np.pi*(r3**2))
    total_area=area1+area2+area3
    n1 = int(n*0.6*area1/total_area)
    n2 = int(n*0.6*area2/total_area)
    n_disk = int(n*0.6*area3/total_area)
    n_noise=n-n1-n2-n_disk

    # Annulus 1 (centered at origin)
    annulus1 = sample_annulus(n1, r_inner=r11, r_outer=r12, noise=noise, center=(-1,-1))

    # Annulus 2 (shifted so it does not overlap)
    annulus2 = sample_annulus(n2, r_inner=r21, r_outer=r22, noise=noise, center=(1,0))

    # Disk (shifted so it does not overlap)
    disk_points = sample_disk(n_disk, radius=r3, noise=noise, center=(-0.8,1.2))
    more_noise= np.random.uniform(low=-2, high=2, size=(n_noise,2))
    # Combine all points
    return np.vstack([annulus1, annulus2, disk_points, more_noise])

def generate_three_annuli(n, r11, r12, r21, r22, r31, r32, noise=0.15):
    """

    Generate points for two non-overlapping annuli and a disk.
    """
    area1=(np.pi*(r12**2-r11**2))
    area2=(np.pi*(r22**2-r21**2))
    area3=(np.pi*(r32**2-r31**2))
    total_area=area1+area2+area3
    n1 = int(n*0.5*area1/total_area)
    n2 = int(n*0.5*area2/total_area)
    n3= int(n*0.5*area3/total_area)
    n_noise=n-n1-n2-n3
    # Annulus 1 (centered at origin)
    annulus1 = sample_annulus(n1, r_inner=r11, r_outer=r12, noise=noise, center=(-1,-1))

    # Annulus 2 (shifted so it does not overlap)
    annulus2 = sample_annulus(n2, r_inner=r21, r_outer=r22, noise=noise, center=(1,0))

    # Disk (shifted so it does not overlap)
    annulus3= sample_annulus(n3, r_inner=r31, r_outer=r32, noise=noise, center=(-0.8,1.2))
    more_noise= np.random.uniform(low=-2, high=2, size=(n_noise,2))
    # Combine all points
    return np.vstack([annulus1, annulus2, annulus3, more_noise])



In [ ]:
points = generate_three_annuli(1000, 0.2,0.45, 0.3, 0.55, 0.7,0.9,noise=0.10)

kde = KernelDensity(kernel="gaussian", bandwidth=0.2)
kde.fit(points)

# 2. Evaluate log density at the same points
log_density = kde.score_samples(points)  # returns log-density

# 3. Convert to normal density if needed
density = np.exp(log_density)

# Create 1 row, 2 columns of plots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# --- Figure 2 (right): simple scatter ---
axes[0].scatter(points[:, 0], points[:, 1], s=5)
axes[0].set_xlim(-2, 2)
axes[0].set_ylim(-2, 2)
axes[0].set_aspect(1)
axes[0].set_title("Scatter plot")

# --- Figure 1 (left): density-colored scatter ---
sc = axes[1].scatter(points[:, 0], points[:, 1], c=-density, cmap="viridis_r")
axes[1].set_aspect(1)
axes[1].set_title(f"Density plot (size {points.shape[0]})")
fig.colorbar(sc, ax=axes[0])



plt.tight_layout()
plt.show()

In [ ]:
#Generate Signed Rank Barcode using sample code from the Multipers GitHub
import multipers as mp
simplextree = RipsLowerstar(points=points, function = 1-density, threshold_radius=2.5)
simplextree.collapse_edges(-2)
simplextree.expansion(2)

simplextree.prune_above_dimension(2)
grid = mp.grids.compute_grid(
    simplextree,
    strategy="regular",
    resolution=50,
)
rank_sm = mp.signed_measure(
    simplextree.grid_squeeze(grid),
    degree=1,
    invariant = "rank",
    plot = True,
);

In [ ]:
#Vectorize this signed barcode using previously defined vectorization map.

#Step 1: Extract and normalize barcode
barcode = rank_sm[0][0]
multiplicities=rank_sm[0][1]

max_val = max(arr[np.isfinite(arr)].max() for arr in barcode)
norm_barcode = barcode / max_val

norm_barcode=replace_inf_safe(norm_barcode)



In [ ]:
MPHvect_visualize(norm_barcode, multiplicities=rank_sm[0][1], number_of_triangulations=3)
